In [1]:
import numpy as np

In [2]:
THETA_MIN      = 2.5            # Minimum angular scale (in arcminutes)
THETA_MAX      = 900.           # Maximum angular scale (in arcminutes)
N_ANGULAR_BINS = 26             # Number of angular bins

N_LENS_BINS    = 5              # Number of lens tomographic bins
N_SRC_BINS     = 5              # Number of source tomographic bins

N_XI_POWERSPECTRA = int(N_SRC_BINS * (N_SRC_BINS + 1) / 2)      # Number of power spectra

In [3]:
theta_bins = np.logspace(np.log10(THETA_MIN), np.log10(THETA_MAX), N_ANGULAR_BINS+1)

#### Mask for cosmic shear ($\xi_{\pm}$)

In [4]:
ξ_plus_CUTOFF  = 2.756             # cutoff scale in arcminutes
ξ_minus_CUTOFF = 8.696             # cutoff scale in arcminutes

ξ_plus_mask  = (theta_bins[:-1] > ξ_plus_CUTOFF)
ξ_minus_mask = (theta_bins[:-1] > ξ_minus_CUTOFF)

ξ_plus_fullmask  = np.hstack([ξ_plus_mask for i in range(N_XI_POWERSPECTRA)])
ξ_minus_fullmask = np.hstack([ξ_minus_mask for i in range(N_XI_POWERSPECTRA)])

#### Create mask for GGL($\gamma_t$) and galaxy clustering ($w$)

In [5]:
lens_cutoff = [80.88, 54.19, 42.85, 35.43, 29.73]       # cutoff scale in arcmins. 
                                                        # Different for different redshift bins
lens_mask_list = [(theta_bins[:-1] > lens_cutoff[i]) for i in range(N_LENS_BINS)]

In [6]:
γt_fullmask_list = []

for i in range(N_LENS_BINS):
    for j in range(N_SRC_BINS):
        lens_mask = lens_mask_list[i]
        γt_fullmask_list.append(lens_mask)
        
γt_fullmask = np.hstack(γt_fullmask_list)        

In [7]:
w_fullmask = np.hstack(lens_mask_list)

#### Combine all the masks

In [8]:
lsst_3x2_mask = np.hstack([ξ_plus_fullmask, ξ_minus_fullmask, γt_fullmask, w_fullmask])

In [9]:
with open('data/lsst_3x2_fiducial_mask.mask', 'w') as f:
    for i, mask_val in enumerate(lsst_3x2_mask):
        f.write("%d\t%d\n"%(i, int(mask_val)))